In [4]:
import os
import glob
from langchain.agents import create_agent

from dotenv import load_dotenv

load_dotenv()

True

In [58]:
referential_path = "referentiel/moon_match"
vectorstore_path = "../vs_referentiel"

sys_prompt_gen_us = """
    Vous êtes un analyste d'affaires expert spécialisé dans la rédaction de récits utilisateur de haute qualité, optimisés pour les tests et l'assurance qualité. Vous comprenez que des récits utilisateur bien rédigés facilitent la création de cas de test exhaustifs et réduisent les ambiguïtés lors du développement.

## Principes fondamentaux

### Structure obligatoire
Chaque récit utilisateur DOIT suivre le format :
- **En tant que** [rôle/persona spécifique]
- **Je veux** [action/fonctionnalité claire et atomique]
- **Afin de** [bénéfice mesurable ou valeur apportée]

### Critères d'acceptation testables
Chaque récit DOIT inclure des critères d'acceptation qui sont :
- **Spécifiques** : Pas d'ambiguïté sur ce qui doit être vérifié
- **Mesurables** : Résultats quantifiables ou observables
- **Vérifiables** : Peuvent être traduits directement en cas de test
- **Complets** : Couvrent le chemin nominal ET les cas d'erreur

### Format des critères d'acceptation
Utilisez le format Given/When/Then (Étant donné/Quand/Alors) :
- **Étant donné** [contexte/précondition]
- **Quand** [action de l'utilisateur]
- **Alors** [résultat attendu observable]

## Exigences pour la testabilité

### Cas à couvrir obligatoirement
1. **Chemin nominal** (happy path) : Le scénario où tout fonctionne correctement
2. **Cas d'erreur** : Comportement en cas d'entrées invalides ou d'erreurs système
3. **Cas limites** : Valeurs aux frontières (min, max, vide, null)
4. **Cas aux limites** : Comportement à la limite exacte des plages valides

### Données de test
- Spécifiez les types de données attendus (format, longueur, caractères autorisés)
- Indiquez les valeurs limites explicitement
- Précisez les dépendances de données entre champs

### Messages et retours utilisateur
- Décrivez le texte exact des messages d'erreur attendus
- Spécifiez où et comment les messages doivent s'afficher
- Indiquez la durée d'affichage si applicable

## Règles de rédaction

### Atomicité
- Un récit = une seule fonctionnalité testable
- Si un récit nécessite plus de 8-10 critères d'acceptation, découpez-le

### Précision
- Évitez les termes vagues : "rapide", "convivial", "facile"
- Utilisez des valeurs concrètes : "en moins de 3 secondes", "en 2 clics maximum"
- Nommez les éléments d'interface avec précision

### Indépendance
- Chaque récit doit être testable indépendamment
- Précisez explicitement les dépendances si elles existent

## Structure de sortie

"""

In [6]:
import getpass

if not os.environ.get("AZURE_OPENAI_API_KEY"):
    os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("Enter API key for Azure: ")

from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

model = AzureChatOpenAI(
    azure_endpoint="https://menoua.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview",
    azure_deployment="gpt-41",
    api_version="2025-01-01-preview",
    temperature=0,
    use_responses_api=False,
)

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    chunk_size=64,
    # max_retries=10,
    # retry_min_seconds=10,
    # retry_max_seconds=60,
)

# BDDv FAISS

In [37]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world")) # 1536
index = faiss.IndexFlatL2(embedding_dim)

vectorstore = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [38]:
from langchain_community.document_loaders import PyPDFLoader

pdf_files = glob.glob(os.path.join(referential_path, "*.pdf"))
all_docs = []

for pdf_path in pdf_files:
    loader = PyPDFLoader(pdf_path)
    # loader = PyPDFLoader(pdf_path, mode="single")
    docs = loader.load()
    all_docs.extend(docs)

In [39]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    # add_start_index=True,
)

all_splits = text_splitter.split_documents(all_docs)

print(f"Répartition du référentiel en {len(all_splits)} sous-documents.")

Répartition du référentiel en 19 sous-documents.


In [40]:
""" si la bdd existe déjà
vectorstore = FAISS.load_local(
    vectorstore_path,
    embeddings,
    allow_dangerous_deserialization=True
)
"""

vectorstore = FAISS.from_documents(documents=all_splits, embedding=embeddings)
vectorstore.save_local(vectorstore_path)

# RAG

In [52]:
from langchain.tools import tool
from typing import Literal

@tool(response_format="content_and_artifact")
def retrieve_context(query: str, section: Literal["Amélioration de récit utilisateur", "Génération de récit utilisateur", "Génération de cas de test", "Autre"]):
    """ajoute du contexte à la requête"""
    retrieved_docs = vectorstore.similarity_search(query, k=2)

    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )

    return serialized, retrieved_docs

In [59]:
from langchain.agents import create_agent

tools = [retrieve_context]

prompt = (
    sys_prompt_gen_us
)
agent = create_agent(model, tools, system_prompt=prompt)

In [60]:
query = (
    "Génère un récit utilisateur pour la Coupe du Monde 2030 sur la Lune.\n"
)

for event in agent.stream(
        {"messages": [{"role": "user", "content": query}]},
        stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Génère un récit utilisateur pour la Coupe du Monde 2030 sur la Lune.

================================== Ai Message ==================================

**Récit utilisateur**

- **En tant que** spectateur accrédité de la Coupe du Monde 2030 sur la Lune
- **Je veux** pouvoir réserver un siège dans le stade lunaire via l’application officielle
- **Afin de** garantir ma place et recevoir toutes les informations nécessaires pour assister au match en toute sécurité

---

**Critères d’acceptation**

1. **Chemin nominal**
   - **Étant donné** que je suis connecté à l’application officielle avec mon compte validé et que le match sélectionné a des places disponibles
   - **Quand** je sélectionne le match et le siège souhaité, puis confirme ma réservation
   - **Alors** ma réservation est enregistrée, je reçois une confirmation affichée à l’écran (“Votre siège A12 pour le match France-Brésil est réservé”) et un emai

## Appel unique

In [61]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """injecte le contexte"""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vectorstore.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        sys_prompt_gen_us +
        f"\n\n{docs_content}"
    )

    return system_message


agent = create_agent(model, tools=[], middleware=[prompt_with_context])

In [62]:
query = "Génère un récit utilisateur pour la Coupe du Monde 2030 sur la Lune.\n"
for step in agent.stream(
        {"messages": [{"role": "user", "content": query}]},
        stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Génère un récit utilisateur pour la Coupe du Monde 2030 sur la Lune.

================================== Ai Message ==================================

### Récit utilisateur

**En tant que** fan de football souhaitant assister à la Coupe du Monde 2030 sur la Lune,  
**Je veux** réserver un billet pour un match spécifique via l’application,  
**Afin de** garantir ma place et recevoir une confirmation immédiate de ma réservation.

---

### Critères d’acceptation

#### Chemin nominal

1. **Étant donné** que je suis connecté à mon compte utilisateur et que la vente des billets est ouverte,  
   **Quand** je sélectionne un match, un stade lunaire, et une catégorie de place disponible, puis que je clique sur "Réserver",  
   **Alors** le système doit afficher un récapitulatif de la réservation (match, date, heure, stade, catégorie, prix) et me demander de confirmer.

2. **Étant donné** que j’ai confirmé la rése